## Airlines Delays using H2O and Python

In [1]:
import h2o
import pandas
import pprint
import operator
import matplotlib
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from tabulate import tabulate

In [9]:
# Connect to a cluster
h2o.init() 
#h2o.shutdown()



No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /var/folders/w_/205bsmdd2tx9ywh2nf36k76h0000gn/T/tmpSxc5HR/h2o_iglopezfrancos_started_from_python.out
JVM stderr: /var/folders/w_/205bsmdd2tx9ywh2nf36k76h0000gn/T/tmppUh1n1/h2o_iglopezfrancos_started_from_python.err
Using ice_root: /var/folders/w_/205bsmdd2tx9ywh2nf36k76h0000gn/T/tmp0d2JZ1


Java Version: java version "1.8.0_40"
Java(TM) SE Runtime Environment (build 1.8.0_40-b27)
Java HotSpot(TM) 64-Bit Server VM (build 25.40-b25, mixed mode)


Starting H2O JVM and connecting: ..................... Connection successful!


H2O cluster uptime:,2 seconds 578 milliseconds
H2O cluster version:,3.8.2.3
H2O cluster name:,H2O_started_from_python_iglopezfrancos_lss849
H2O cluster total nodes:,1
H2O cluster total free memory:,1.78 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


Check H2O Flow: http://localhost:54321/flow/index.html

In [10]:
# set this to True if interactive (matplotlib) plots are desired
interactive = False
if not interactive: matplotlib.use('Agg', warn=False)
import matplotlib.pyplot as plt

### 1- Load data

Get the data [here](https://raw.github.com/0xdata/h2o/master/smalldata/airlines/allyears2k_headers.zip)

- 1 row per flight.  
- Has columns showing the origin, destination, departure and arrival time, carrier information, and whether the flight was delayed.

**Pass a panda DataFrame to a H2Oframe:**

*it losses the headers when importing. Column names will be named: C1, C2,..*

In [67]:
import pandas as pd
air_pd = pd.read_csv("allyears2k_headers.csv", low_memory=False)

air = h2o.H2OFrame(air_pd)  


Parse Progress: [##################################################] 100%


In [75]:
column_names = list(air_pd.columns.values)
air.col_names = column_names  #assign column_names to the H2Oframe columns

**Convert H2Oframe to a python object:**

In [66]:
# To convert H2Oframe to a list:
air_as_list = h2o.as_list(air, use_pandas=False) 
air_as_list[0] # header
air_as_list[1] # first row of the H2Oframe

In [76]:
# To convert H2Oframe to a panda data frame:
air_as_pandas = h2o.as_list(air) 
air_as_pandas.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
0,1987,10,14,3,741.0,730,912.0,849,PS,1451,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,YES,YES
1,1987,10,15,4,729.0,730,903.0,849,PS,1451,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,YES,NO
2,1987,10,17,6,741.0,730,918.0,849,PS,1451,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,YES,YES
3,1987,10,18,7,729.0,730,847.0,849,PS,1451,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NO,NO
4,1987,10,19,1,749.0,730,922.0,849,PS,1451,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,YES,YES


In [34]:
from h2o.utils.shared_utils import _locate # private function. used to find files within h2o git project directory.
air_path = [_locate("./allyears2k_headers.csv")]

print("Import and Parse airlines data")
data = h2o.import_file(path=air_path)
data.describe()

Import and Parse airlines data

Parse Progress: [##################################################] 100%
Rows:43,978 Cols:31

Chunk compression summary: 


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C0L,Constant Integers,38,7.6612906,3.0 KB,0.2208526
C0D,Constant Reals,75,15.120968,5.9 KB,0.4358932
CBS,Bits,32,6.451613,12.9 KB,0.9624521
CX0,Zero Sparse Bits,32,6.451613,4.6 KB,0.3456633
CXI,Zero Sparse Integers,7,1.4112903,5.4 KB,0.4017482
C1,1-Byte Integers,12,2.4193547,33.3 KB,2.4750016
C1N,1-Byte Integers (w/o NAs),99,19.959679,271.9 KB,20.22508
C1S,1-Byte Fractions,26,5.2419353,72.1 KB,5.360106
C2,2-Byte Integers,172,34.677418,934.3 KB,69.50506
CNAXI,NA Sparse Integers,3,0.6048387,938 B,0.0681446



Frame distribution summary: 


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
127.0.0.1:54321,1.3 MB,43978.0,16.0,496.0
mean,1.3 MB,43978.0,16.0,496.0
min,1.3 MB,43978.0,16.0,496.0
max,1.3 MB,43978.0,16.0,496.0
stddev,0 B,0.0,0.0,0.0
total,1.3 MB,43978.0,16.0,496.0


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,16.0,17.0,14.0,-63.0,-16.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1997.5,1.40909090909,14.6010732639,3.82061485288,1345.84666138,1313.22286143,1504.63413038,1485.28916731,NaN,818.842989677,NaN,124.814529135,125.021562607,114.316111091,9.31711193698,10.0073906556,NaN,NaN,730.182190565,5.38136805953,14.1686341847,0.0246941652645,NaN,0.00247851198326,4.04780029106,0.289376469271,4.85503190418,0.0170155602821,7.62006045002,0.555755150302,0.525057983537
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,9.0,3949.0,3500.0,475.0,437.0,402.0,475.0,473.0,131.0,133.0,3365.0,128.0,254.0,1.0,3.0,1.0,369.0,201.0,323.0,14.0,373.0,1.0,1.0
sigma,6.34436090171,1.87471137134,9.17579042586,1.90501311913,465.340899124,476.251139993,484.347487904,492.750434123,NaN,777.404369164,NaN,73.9744416606,73.40159463,69.6363295151,29.8402219624,26.4388090429,NaN,NaN,578.43800823,4.20197993986,9.9050857472,0.155193141358,NaN,0.0497234872189,16.2057299045,4.41677989873,18.6197762215,0.403940182102,23.4875658741,0.496887288343,0.499377380318
zeros,0,0,0,0,0,569,0,569,724,0,2,0,0,0,1514,6393,59,172,0,623,557,42892,81,43869,7344,8840,7388,8914,7140,19537,20887
missing,0,0,0,0,1086,0,1195,0,0,0,32,1195,13,16649,1195,1086,0,0,35,16026,16024,0,9774,0,35045,35045,35045,35045,35045,0,0
0,1987.0,10.0,14.0,3.0,741.0,730.0,912.0,849.0,PS,1451.0,NA,91.0,79.0,nan,23.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES
1,1987.0,10.0,15.0,4.0,729.0,730.0,903.0,849.0,PS,1451.0,NA,94.0,79.0,nan,14.0,-1.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,NO
2,1987.0,10.0,17.0,6.0,741.0,730.0,918.0,849.0,PS,1451.0,NA,97.0,79.0,nan,29.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES


### 2- Data exploration and munging. 
- Generate scatter plots of various columns and plot fitted GLM model.

In [35]:
# Function to fit a GLM model and plot the fitted (x,y) values
def scatter_plot(data, x, y, max_points = 1000, fit = True):
    if(fit):
        lr = H2OGeneralizedLinearEstimator(family = "gaussian")
        lr.train(x=x, y=y, training_frame=data)
        coeff = lr.coef()
    df = data[[x,y]]
    runif = df[y].runif()
    df_subset = df[runif < float(max_points)/data.nrow]
    df_py = h2o.as_list(df_subset)
    
    if(fit): h2o.remove(lr._id)

    # If x variable is string, generate box-and-whisker plot
    if(df_py[x].dtype == "object"):
        if interactive: df_py.boxplot(column = y, by = x)
    # Otherwise, generate a scatter plot
    else:
        if interactive: df_py.plot(x = x, y = y, kind = "scatter")
    
    if(fit):
        x_min = min(df_py[x])
        x_max = max(df_py[x])
        y_min = coeff["Intercept"] + coeff[x]*x_min
        y_max = coeff["Intercept"] + coeff[x]*x_max
        plt.plot([x_min, x_max], [y_min, y_max], "k-")
    if interactive: plt.show()

scatter_plot(data, "Distance", "AirTime", fit = True)
scatter_plot(data, "UniqueCarrier", "ArrDelay", max_points = 5000, fit = False)


glm Model Build Progress: [##################################################] 100%


Remove rows where departure delay is NA.

In [36]:
data = data[data["DepDelay"].isna()==0]
rows, cols = data.dim
print("New dataset rows {0}, columns: {1}".format(rows, cols))

New dataset rows 42892, columns: 31


Create a new target variable called **`IsDepDelayed25`** that identifies those flights that were delayed for more than 15 min.

In [47]:
data1 = data
numCols = data1.dim[1]
new_y_col = "IsDepDelayed25"

minOfDelayThreshold = 25
data1 = data1.cbind(data1["DepDelay"] > minOfDelayThreshold)


data1[numCols] = data1[numCols-1].asfactor()
data1.set_name(numCols, new_y_col)
data = data1

In [48]:
data.head()

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed,SynthDepDelayed,IsDepDelayed15,IsDepDelayed25
1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,nan,23,11,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES,YES,YES,YES
1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,nan,14,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO,NO,NO,NO
1987,10,17,6,741,730,918,849,PS,1451,NA,97,79,nan,29,11,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES,YES,YES,YES
1987,10,18,7,729,730,847,849,PS,1451,NA,78,79,nan,-2,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,NO,NO,NO,NO,NO
1987,10,19,1,749,730,922,849,PS,1451,NA,93,79,nan,33,19,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES,YES,YES,YES
1987,10,21,3,728,730,848,849,PS,1451,NA,80,79,nan,-1,-2,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,NO,NO,NO,NO,NO
1987,10,22,4,728,730,852,849,PS,1451,NA,84,79,nan,3,-2,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO,NO,NO,NO
1987,10,23,5,731,730,902,849,PS,1451,NA,91,79,nan,13,1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES,YES,YES,YES
1987,10,24,6,744,730,908,849,PS,1451,NA,84,79,nan,19,14,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES,YES,YES,YES
1987,10,25,7,729,730,851,849,PS,1451,NA,82,79,nan,2,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO,NO,NO,NO


In [13]:
# Group flights by month
grouped = data.group_by("Month")
bpd = grouped.count().sum("Cancelled").frame
bpd.show()
bpd.describe()
bpd.dim

Month,sum_Cancelled,nrow_Year
1,1067,41979
10,19,1999


Rows:2 Cols:3

Chunk compression summary: 


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C1N,1-Byte Integers (w/o NAs),1,33.333336,70 B,30.434782
C2,2-Byte Integers,1,33.333336,72 B,31.304348
C2S,2-Byte Fractions,1,33.333336,88 B,38.260868



Frame distribution summary: 


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
127.0.0.1:54321,230 B,2.0,1.0,3.0
mean,230 B,2.0,1.0,3.0
min,230 B,2.0,1.0,3.0
max,230 B,2.0,1.0,3.0
stddev,0 B,0.0,0.0,0.0
total,230 B,2.0,1.0,3.0


,Month,sum_Cancelled,nrow_Year
type,int,int,int
mins,1.0,19.0,1999.0
mean,5.5,543.0,21989.0
maxs,10.0,1067.0,41979.0
sigma,6.36396103068,741.047906684,28270.1291118
zeros,0,0,0
missing,0,0,0
0,1.0,1067.0,41979.0
1,10.0,19.0,1999.0


[2, 3]

In [9]:
# Convert columns to factors
data["Year"]      = data["Year"]     .asfactor()
data["Month"]     = data["Month"]    .asfactor()
data["DayOfWeek"] = data["DayOfWeek"].asfactor()
data["Cancelled"] = data["Cancelled"].asfactor()

In [16]:
# Calculate and plot travel time
hour1 = data["CRSArrTime"] / 100    # divide by 100, return no decimal points.
mins1 = data["CRSArrTime"] % 100    # 2 decimal points
arrTime = hour1*60 + mins1
arrTime.head(1)

CRSArrTime
558.4


In [17]:
hour2 = data["CRSDepTime"] / 100
mins2 = data["CRSDepTime"] % 100
depTime = hour2*60 + mins2
depTime.head(1)

CRSDepTime
468


In [18]:
# TODO: Replace this once list comprehension is supported. See PUBDEV-1286.
# data["TravelTime"] = [x if x > 0 else None for x in (arrTime - depTime)]
data["TravelTime"] = (arrTime-depTime > 0).ifelse((arrTime-depTime), h2o.H2OFrame([[None]] * data.nrow))
scatter_plot(data, "Distance", "TravelTime")


Parse Progress: [##################################################] 100%

glm Model Build Progress: [##################################################] 100%


In [20]:
# Impute missing travel times and re-plot
data.impute(column = "Distance", by = ["Origin", "Dest"])
scatter_plot(data, "Distance", "TravelTime")


glm Model Build Progress: [##################################################] 100%


### 3- Fit a model on train; using test as validation

In [21]:
# Create test/train split
s = data["Year"].runif()
train = data[s <= 0.75]
test  = data[s > 0.75]

In [22]:
# Set predictor and response variables
myY = "IsDepDelayed"
myX = ["Origin", "Dest", "Year", "UniqueCarrier", "DayOfWeek", "Month", "Distance", "FlightNum"]

#### GLM

In [24]:
# Simple GLM - Predict Delays
data_glm = H2OGeneralizedLinearEstimator(family="binomial", standardize=True)
data_glm.train(x               =myX,
               y               =myY,
               training_frame  =train,
               validation_frame=test)


glm Model Build Progress: [##################################################] 100%


#### GBM

In [26]:
# Simple GBM
data_gbm = H2OGradientBoostingEstimator(balance_classes=True,
                                        ntrees         =3,
                                        max_depth      =1,
                                        distribution   ="bernoulli",
                                        learn_rate     =0.1,
                                        min_rows       =2)
data_gbm.train(x               =myX,
               y               =myY,
               training_frame  =train,
               validation_frame=test)


gbm Model Build Progress: [##################################################] 100%


In [27]:
# Complex GBM
data_gbm2 = H2OGradientBoostingEstimator(balance_classes=True,
                                         ntrees         =50,           #Increase 
                                         max_depth      =5,            #Increase
                                         distribution   ="bernoulli",
                                         learn_rate     =0.1,
                                         min_rows       =2)

data_gbm2.train(x               =myX,
                y               =myY,
                training_frame  =train,
                validation_frame=test)


gbm Model Build Progress: [##################################################] 100%


#### Random Forest

In [28]:
# Simple Random Forest
data_rf = H2ORandomForestEstimator(ntrees         =5,
                                   max_depth      =2,
                                   balance_classes=True)

data_rf.train(x               =myX,
              y               =myY,
              training_frame  =train,
              validation_frame=test)


drf Model Build Progress: [##################################################] 100%


In [29]:
# Complex Random Forest
data_rf2 = H2ORandomForestEstimator(ntrees         =10,
                                    max_depth      =5,
                                    balance_classes=True)

data_rf2.train(x               =myX,
               y               =myY,
               training_frame  =train,
               validation_frame=test)


drf Model Build Progress: [##################################################] 100%


#### Deep Learning

In [30]:
# Deep Learning with 5 epochs
data_dl = H2ODeepLearningEstimator(hidden              =[10,10],
                                   epochs              =5,
                                   variable_importances=True,
                                   balance_classes     =True,
                                   loss                ="Automatic")

data_dl.train(x               =myX,
              y               =myY,
              training_frame  =train,
              validation_frame=test)


deeplearning Model Build Progress: [##################################################] 100%


### 4- Model Evaluation

Variable Importance

#### GLM:

In [31]:
# Variable importances from each algorithm
# Calculate magnitude of normalized GLM coefficients
from six import iteritems
glm_varimp = data_glm.coef_norm()
for k,v in iteritems(glm_varimp):
    glm_varimp[k] = abs(glm_varimp[k])

In [32]:
# Sort in descending order by magnitude
glm_sorted = sorted(glm_varimp.items(), key = operator.itemgetter(1), reverse = True)
table = tabulate(glm_sorted, headers = ["Predictor", "Normalized Coefficient"], tablefmt = "orgtbl")
print("Variable Importances:\n\n" + table)

Variable Importances:

| Predictor                 |   Normalized Coefficient |
|---------------------------+--------------------------|
| Year.2008                 |              2.1131      |
| Origin.MDW                |              1.63015     |
| Origin.HPN                |              1.59973     |
| Year.2003                 |              1.55188     |
| Origin.LIH                |              1.48413     |
| Year.2007                 |              1.37056     |
| Dest.LYH                  |              1.2822      |
| Origin.GRR                |              1.27255     |
| UniqueCarrier.TW          |              1.21211     |
| UniqueCarrier.HP          |              1.16888     |
| Origin.TLH                |              1.15401     |
| Origin.CAE                |              1.14564     |
| Origin.LEX                |              1.10873     |
| Dest.HTS                  |              1.10631     |
| Origin.MYR                |              1.10127     |
| Origin

#### GBM

varimp ranks features by importance in the model and returns 3 numbers for every feature:
- variable relative_importance 
- scaled_importance 
- percentage

In [48]:
print tabulate(data_gbm.varimp(), headers = ["Predictor", "relative_importance", "scaled_importance", "percentage"], tablefmt = "orgtbl")

| Predictor     |   relative_importance |   scaled_importance |   percentage |
|---------------+-----------------------+---------------------+--------------|
| Year          |               1265.57 |                   1 |            1 |
| Month         |                  0    |                   0 |            0 |
| DayOfWeek     |                  0    |                   0 |            0 |
| UniqueCarrier |                  0    |                   0 |            0 |
| FlightNum     |                  0    |                   0 |            0 |
| Origin        |                  0    |                   0 |            0 |
| Dest          |                  0    |                   0 |            0 |
| Distance      |                  0    |                   0 |            0 |


In [49]:
print tabulate(data_gbm2.varimp(), headers = ["Predictor", "relative_importance", "scaled_importance", "percentage"], tablefmt = "orgtbl")

| Predictor     |   relative_importance |   scaled_importance |   percentage |
|---------------+-----------------------+---------------------+--------------|
| Origin        |              5002.48  |           1         |   0.370954   |
| Year          |              4320.86  |           0.863744  |   0.320409   |
| Dest          |              3028.85  |           0.605469  |   0.224601   |
| UniqueCarrier |               416.63  |           0.0832848 |   0.0308948  |
| DayOfWeek     |               393.034 |           0.0785679 |   0.0291451  |
| FlightNum     |               206.998 |           0.0413791 |   0.0153498  |
| Distance      |               116.585 |           0.0233054 |   0.00864525 |
| Month         |                 0     |           0         |   0          |


#### RF

In [50]:
print tabulate(data_rf.varimp(), headers = ["Predictor", "relative_importance", "scaled_importance", "percentage"], tablefmt = "orgtbl")

| Predictor     |   relative_importance |   scaled_importance |   percentage |
|---------------+-----------------------+---------------------+--------------|
| Year          |            1137.81    |          1          |   0.575161   |
| Origin        |             540.45    |          0.474992   |   0.273197   |
| FlightNum     |             130.724   |          0.114891   |   0.0660809  |
| UniqueCarrier |              62.9088  |          0.0552895  |   0.0318004  |
| DayOfWeek     |              59.4246  |          0.0522273  |   0.0300391  |
| Dest          |              40.8272  |          0.0358823  |   0.0206381  |
| Distance      |               6.10068 |          0.00536179 |   0.00308389 |
| Month         |               0       |          0          |   0          |


In [51]:
print tabulate(data_rf2.varimp(), headers = ["Predictor", "relative_importance", "scaled_importance", "percentage"], tablefmt = "orgtbl")

| Predictor     |   relative_importance |   scaled_importance |   percentage |
|---------------+-----------------------+---------------------+--------------|
| Origin        |             2765.45   |          1          |   0.347586   |
| Year          |             2290.27   |          0.828171   |   0.287861   |
| Dest          |             1647.37   |          0.595698   |   0.207057   |
| UniqueCarrier |              583.46   |          0.210982   |   0.0733345  |
| FlightNum     |              384.383  |          0.138995   |   0.0483127  |
| DayOfWeek     |              175.07   |          0.0633062  |   0.0220044  |
| Distance      |               94.9799 |          0.0343452  |   0.0119379  |
| Month         |               15.1693 |          0.00548528 |   0.00190661 |


Model Performance on test data

**GLM**

In [52]:
data_glm.model_performance(test)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.215320317858
R^2: 0.137128283161
LogLoss: 0.619436244795
Null degrees of freedom: 10993
Residual degrees of freedom: 10773
Null deviance: 15221.6608227
Residual deviance: 13620.1641505
AIC: 14062.1641505
AUC: 0.713829832276
Gini: 0.427659664551

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.357488777917: 


,NO,YES,Error,Rate
NO,1686.0,3575.0,0.6795,(3575.0/5261.0)
YES,675.0,5058.0,0.1177,(675.0/5733.0)
Total,2361.0,8633.0,0.3866,(4250.0/10994.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3574888,0.7041626,288.0
max f2,0.1383360,0.8458469,384.0
max f0point5,0.5598263,0.6802843,178.0
max accuracy,0.4976592,0.6644533,211.0
max precision,0.9832504,1.0,0.0
max recall,0.0779967,1.0,398.0
max specificity,0.9832504,1.0,0.0
max absolute_MCC,0.5421518,0.3265985,187.0
max min_per_class_accuracy,0.5226907,0.6605208,198.0



Gains/Lift Table: Avg response rate: 52.15 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100055,0.9198348,1.8653696,1.8653696,0.9727273,0.9727273,0.0186639,0.0186639,86.5369551,86.5369551
,2,0.0200109,0.8958523,1.7084693,1.7869194,0.8909091,0.9318182,0.0170940,0.0357579,70.8469308,78.6919430
,3,0.0300164,0.8739085,1.6736026,1.7491471,0.8727273,0.9121212,0.0167452,0.0525031,67.3602588,74.9147149
,4,0.0400218,0.8595675,1.6213025,1.7171860,0.8454545,0.8954545,0.0162219,0.0687249,62.1302507,71.7185989
,5,0.0500273,0.8463910,1.5690024,1.6875493,0.8181818,0.88,0.0156986,0.0844235,56.9002426,68.7549276
,6,0.1000546,0.7880522,1.5550557,1.6213025,0.8109091,0.8454545,0.0777952,0.1622187,55.5055738,62.1302507
,7,0.1500819,0.7332656,1.4121022,1.5515691,0.7363636,0.8090909,0.0706436,0.2328624,41.2102184,55.1569066
,8,0.2000182,0.6937665,1.3413208,1.4990787,0.6994536,0.7817190,0.0669806,0.2998430,34.1320835,49.9078716
,9,0.3000728,0.6383579,1.2987853,1.4322940,0.6772727,0.7468930,0.1299494,0.4297924,29.8785342,43.2294020
,10,0.4000364,0.5807701,1.2109761,1.3769897,0.6314832,0.7180537,0.1210535,0.5508460,21.0976092,37.6989699


**GBM2**

In [59]:
# Model performance of GBM model on test data
data_gbm2.model_performance(test)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.207509711429
R^2: 0.168428401266
LogLoss: 0.602440610978
AUC: 0.737145793355
Gini: 0.474291586709

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.395996389793: 


,NO,YES,Error,Rate
NO,2405.0,2856.0,0.5429,(2856.0/5261.0)
YES,928.0,4805.0,0.1619,(928.0/5733.0)
Total,3333.0,7661.0,0.3442,(3784.0/10994.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3959964,0.7174854,261.0
max f2,0.1450753,0.8460515,385.0
max f0point5,0.5580366,0.6999330,179.0
max accuracy,0.4905562,0.6808259,212.0
max precision,0.9620279,1.0,0.0
max recall,0.0988716,1.0,397.0
max specificity,0.9620279,1.0,0.0
max absolute_MCC,0.5580366,0.3631585,179.0
max min_per_class_accuracy,0.5166197,0.6789584,199.0



Gains/Lift Table: Avg response rate: 52.15 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100964,0.9214375,1.7967355,1.7967355,0.9369369,0.9369369,0.0181406,0.0181406,79.6735511,79.6735511
,2,0.0200109,0.8939416,1.6185835,1.7084693,0.8440367,0.8909091,0.0160474,0.0341880,61.8583543,70.8469308
,3,0.0300164,0.8837243,1.7607694,1.7259027,0.9181818,0.9,0.0176173,0.0518053,76.0769389,72.5902669
,4,0.0402037,0.8707860,1.7464491,1.7311090,0.9107143,0.9027149,0.0177917,0.0695971,74.6449129,73.1109012
,5,0.0502092,0.8563083,1.6387359,1.7127013,0.8545455,0.8931159,0.0163963,0.0859934,63.8735867,71.2701320
,6,0.1001455,0.8017461,1.6067906,1.6598902,0.8378871,0.8655767,0.0802372,0.1662306,60.6790584,65.9890245
,7,0.1499909,0.7545543,1.4452510,1.5885608,0.7536496,0.8283808,0.0720391,0.2382697,44.5251018,58.8560775
,8,0.2002911,0.7239732,1.4009738,1.5414511,0.7305606,0.8038147,0.0704692,0.3087389,40.0973836,54.1451067
,9,0.3000728,0.6591716,1.3635208,1.4822848,0.7110301,0.7729615,0.1360544,0.4447933,36.3520796,48.2284802
,10,0.4001273,0.5869353,1.2325386,1.4198341,0.6427273,0.7403955,0.1233211,0.5681144,23.2538573,41.9834051


**RF2**

In [57]:
# Model performance of RF model on test data
data_rf2.model_performance(test)


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.217667126881
R^2: 0.127723712561
LogLoss: 0.625360569476
AUC: 0.712806285986
Gini: 0.425612571973

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.421544423112: 


,NO,YES,Error,Rate
NO,1888.0,3373.0,0.6411,(3373.0/5261.0)
YES,742.0,4991.0,0.1294,(742.0/5733.0)
Total,2630.0,8364.0,0.3743,(4115.0/10994.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4215444,0.7080939,295.0
max f2,0.2536380,0.8456947,388.0
max f0point5,0.5396780,0.6788551,205.0
max accuracy,0.5229355,0.6631799,218.0
max precision,0.9097888,1.0,0.0
max recall,0.2228195,1.0,396.0
max specificity,0.9097888,1.0,0.0
max absolute_MCC,0.5262250,0.3261935,216.0
max min_per_class_accuracy,0.5229355,0.6624804,218.0



Gains/Lift Table: Avg response rate: 52.15 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100055,0.8628798,1.6910359,1.6910359,0.8818182,0.8818182,0.0169196,0.0169196,69.1035948,69.1035948
,2,0.0200109,0.8436920,1.6561692,1.6736026,0.8636364,0.8727273,0.0165707,0.0334903,65.6169228,67.3602588
,3,0.0300164,0.8156747,1.6213025,1.6561692,0.8454545,0.8636364,0.0162219,0.0497122,62.1302507,65.6169228
,4,0.0402037,0.7971578,1.7122050,1.6703683,0.8928571,0.8710407,0.0174429,0.0671551,71.2205029,67.0368345
,5,0.0500273,0.7822033,1.6868390,1.6736026,0.8796296,0.8727273,0.0165707,0.0837258,68.6839028,67.3602588
,6,0.1000546,0.7331320,1.5271624,1.6003825,0.7963636,0.8345455,0.0763998,0.1601256,52.7162361,60.0382475
,7,0.1503547,0.6932084,1.4113771,1.5371520,0.7359855,0.8015729,0.0709925,0.2311181,41.1377107,53.7152004
,8,0.2001092,0.6687118,1.3392135,1.4879373,0.6983547,0.7759091,0.0666318,0.2977499,33.9213527,48.7937301
,9,0.3000728,0.6257869,1.3174164,1.4311314,0.6869882,0.7462868,0.1316937,0.4294436,31.7416353,43.1131444
,10,0.4003093,0.5676497,1.1937580,1.3716937,0.6225045,0.7152920,0.1196581,0.5491017,19.3758047,37.1693706


#### DL

In [60]:
data_dl.model_performance(test)


ModelMetricsBinomial: deeplearning
** Reported on test data. **

MSE: 0.215124286291
R^2: 0.1379138574
LogLoss: 0.619287671853
AUC: 0.71465728299
Gini: 0.429314565981

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.378257191465: 


,NO,YES,Error,Rate
NO,1930.0,3331.0,0.6331,(3331.0/5261.0)
YES,777.0,4956.0,0.1355,(777.0/5733.0)
Total,2707.0,8287.0,0.3737,(4108.0/10994.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3782572,0.7069900,281.0
max f2,0.1431398,0.8455969,388.0
max f0point5,0.5371382,0.6841174,193.0
max accuracy,0.5285964,0.6659996,198.0
max precision,0.9641376,1.0,0.0
max recall,0.0911536,1.0,399.0
max specificity,0.9641376,1.0,0.0
max absolute_MCC,0.5285964,0.3346275,198.0
max min_per_class_accuracy,0.5134259,0.6626117,207.0



Gains/Lift Table: Avg response rate: 52.15 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100055,0.8971412,1.8130695,1.8130695,0.9454545,0.9454545,0.0181406,0.0181406,81.3069470,81.3069470
,2,0.0200109,0.8724379,1.6910359,1.7520527,0.8818182,0.9136364,0.0169196,0.0350602,69.1035948,75.2052709
,3,0.0300164,0.8526134,1.6038691,1.7026582,0.8363636,0.8878788,0.0160474,0.0511076,60.3869147,70.2658188
,4,0.0400218,0.8366343,1.6387359,1.6866776,0.8545455,0.8795455,0.0163963,0.0675039,63.8735867,68.6677608
,5,0.0500273,0.8205518,1.6213025,1.6736026,0.8454545,0.8727273,0.0162219,0.0837258,62.1302507,67.3602588
,6,0.1000546,0.7614338,1.5376224,1.6056125,0.8018182,0.8372727,0.0769231,0.1606489,53.7622378,60.5612483
,7,0.1500819,0.7203978,1.4330222,1.5480824,0.7472727,0.8072727,0.0716902,0.2323391,43.3022216,54.8082394
,8,0.2000182,0.6899574,1.3203627,1.4912301,0.6885246,0.7776262,0.0659341,0.2982732,32.0362697,49.1230137
,9,0.3001637,0.6295033,1.2923804,1.4248866,0.6739328,0.7430303,0.1294261,0.4276993,29.2380442,42.4886648
,10,0.4000364,0.5690637,1.2522487,1.3817861,0.6530055,0.7205548,0.1250654,0.5527647,25.2248749,38.1786054
